In [1]:
import numpy as np
import pandas as pd

#### import all libarary to process the json, plot the data etc

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from geopy.geocoders import Nominatim
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### get the longitude and latitude of London

In [58]:
address = 'london'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 51.5073219, -0.1276474.


### We will pickup some the neighborhood from the wikipedia 
### https://en.wikipedia.org/wiki/Ethnic_groups_in_London
### we will be fetching this ethnic populatin below in the code
### foloowing aare the neighborhoods from above html

In [59]:
#n_list=['Islington','Knightsbridge','Lambeth','Maida Vale','Marylebone','Mayfair','Notting Hill','Chelsea','Canonbury','Bloomsbury','Bermondsey','Brixton','City of London','Fulham','Greenwich']
n_list=['Newham', 'Redbridge', 'Brent',  'Harrow', 'Ealing', 'Hounslow', 'Hillingdon', 'Barnet', 'Croydon', 
        'Merton', 'Camden', 'Enfield', 'Wandsworth',
 'Westminster', 'Greenwich',  'Southwark']

### get the location coordinates of all the neighborhoods.

In [62]:
df1=pd.DataFrame(columns=['neighborhood','latitude','longitude'])
for i in n_list:
    address = i

    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    lat = location.latitude
    lon = location.longitude
    print('The geograpical coordinate of {} are {}, {}.'.format(i,lat, lon))
    df1=df1.append([{'neighborhood':i,'latitude':lat,'longitude':lon}],ignore_index=True)    

The geograpical coordinate of Newham are 51.52999955, 0.02931796029382208.
The geograpical coordinate of Redbridge are 51.5763203, 0.0454097.
The geograpical coordinate of Brent are 32.9373463, -87.1647184.
The geograpical coordinate of Harrow are 51.5967688, -0.33727515543507014.
The geograpical coordinate of Ealing are 51.5126553, -0.3051952.
The geograpical coordinate of Hounslow are 51.4686132, -0.3613471.
The geograpical coordinate of Hillingdon are 51.542519299999995, -0.44833493117949663.
The geograpical coordinate of Barnet are 51.65309, -0.2002261.
The geograpical coordinate of Croydon are 51.3713049, -0.101957.
The geograpical coordinate of Merton are 51.410802849999996, -0.18809850595572736.
The geograpical coordinate of Camden are 39.9448402, -75.1198911.
The geograpical coordinate of Enfield are 51.6520851, -0.0810175.
The geograpical coordinate of Wandsworth are 51.4570271, -0.1932607.
The geograpical coordinate of Westminster are 39.8366528, -105.0372046.
The geograpical

### rename the columns to be refrenced

In [63]:
df=df1.rename(columns={'neighborhood':'Neighborhood','latitude':'Latitude','longitude':'Longitude'})

### fetch the population data from wikipedia to get the enthnic population (which we will se to get India population) in each neighborhood

In [64]:
indian_pop1=pd.read_html("https://en.wikipedia.org/wiki/Ethnic_groups_in_London")

In [65]:
type(indian_pop1)

list

### data for ethnic population is in 4th field of indian_pop1 obejct

In [11]:
indian_pop=pd.DataFrame(indian_pop1[4])

In [66]:
indian_pop.rename(columns={'London Borough':'Neighborhood'},inplace=True)

### rename the column with same as our location data dataframe

In [67]:
temp=indian_pop[['Indian Population','Neighborhood']]

In [68]:
temp

,Indian Population,Neighborhood
0,42484,Newham
1,45660,Redbridge
2,58017,Brent
3,6787,Tower Hamlets
4,63051,Harrow
5,48240,Ealing
6,48161,Hounslow
7,36795,Hillingdon
8,27920,Barnet
9,24660,Croydon


In [69]:
df=pd.merge(df,indian_pop[['Indian Population','Neighborhood']],on='Neighborhood')

### merged the above dataframe to get population on Indians in same dataframe as in location

In [70]:
df

,Neighborhood,Latitude,Longitude,Indian Population
0,Newham,51.530000,0.029318,42484
1,Redbridge,51.576320,0.045410,45660
2,Brent,32.937346,-87.164718,58017
3,Harrow,51.596769,-0.337275,63051
4,Ealing,51.512655,-0.305195,48240
5,Hounslow,51.468613,-0.361347,48161
6,Hillingdon,51.542519,-0.448335,36795
7,Barnet,51.653090,-0.200226,27920
8,Croydon,51.371305,-0.101957,24660
9,Merton,51.410803,-0.188099,8106


In [71]:
london_data=df[['Neighborhood','Latitude','Longitude']]

#### plot London using Folium

In [72]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(london_data['Latitude'], london_data['Longitude'], london_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

#### importing my clietnt id and secret key. After project, I will remove thvalues of these from here

In [73]:
CLIENT_ID='your Id'
CLIENT_SECRET='Your key'

#### define function to get the venues from foursquare api 

In [74]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT=1900
    venues_list=[]
    VERSION='202006101'
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### get all venues for our postal code having in our Dataframe

In [79]:

london_venues = getNearbyVenues(names=london_data['Neighborhood'],
                                   latitudes=london_data['Latitude'],
                                   longitudes=london_data['Longitude']
                                  )



Newham
Redbridge
Brent
Harrow
Ealing
Hounslow
Hillingdon
Barnet
Croydon
Merton
Camden
Enfield
Wandsworth
Westminster
Greenwich
Southwark


In [80]:
london_venues.shape

(521, 7)

In [81]:
london_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Newham,51.53,0.029318,The Black Lion,51.530196,0.022360,Pub
1,Newham,51.53,0.029318,Delicious Café,51.526417,0.030133,Café
2,Newham,51.53,0.029318,Plaistow Park,51.526742,0.025578,Park
3,Newham,51.53,0.029318,KFC,51.531961,0.035439,Fast Food Restaurant
4,Newham,51.53,0.029318,Ginny's Pie and Mash,51.525705,0.029532,Café


#### check the counts of venues for each neighborhood

In [82]:
london_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Barnet,33,33,33,33,33,33
Brent,1,1,1,1,1,1
Camden,20,20,20,20,20,20
Croydon,24,24,24,24,24,24
Ealing,94,94,94,94,94,94
Enfield,62,62,62,62,62,62
Greenwich,57,57,57,57,57,57
Harrow,8,8,8,8,8,8
Hillingdon,4,4,4,4,4,4


In [83]:
print('There are {} uniques categories.'.format(len(london_venues['Venue Category'].unique())))

There are 139 uniques categories.


In [84]:
london_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Newham,51.53,0.029318,The Black Lion,51.530196,0.022360,Pub
1,Newham,51.53,0.029318,Delicious Café,51.526417,0.030133,Café
2,Newham,51.53,0.029318,Plaistow Park,51.526742,0.025578,Park
3,Newham,51.53,0.029318,KFC,51.531961,0.035439,Fast Food Restaurant
4,Newham,51.53,0.029318,Ginny's Pie and Mash,51.525705,0.029532,Café


#### lets do one-hot encoding on our venues to get them in a format which will be used by oour model

In [116]:
# one hot encoding
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
london_onehot['Neighborhood'] = london_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]

london_onehot.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Asian Restaurant,Bagel Shop,Bakery,Bank,Bar,Beach,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Bus Stop,Café,Caribbean Restaurant,Chaat Place,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Convenience Store,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Farm,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,Food Truck,French Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Home Service,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Light Rail Station,Malay Restaurant,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Modern European Restaurant,Multiplex,Museum,New American Restaurant,Nightclub,Noodle House,Optical Shop,Organic Grocery,Outdoor Supply Store,Outlet Store,Park,Performing Arts Venue,Pharmacy,Pie Shop,Pier,Pizza Place,Platform,Playground,Polish Restaurant,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,River,Salad Place,Sandwich Place,Scenic Lookout,Shopping Mall,Southern / Soul Food Restaurant,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Toll Plaza,Tourist Information Center,Train Station,Tram Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Newham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Newham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Newham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Newham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Newham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [117]:
london_onehot.shape

(521, 140)

#### data transformation to assign feature values so that knn can be applied

In [118]:
london_grouped = london_onehot.groupby('Neighborhood').mean().reset_index()
london_grouped.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Asian Restaurant,Bagel Shop,Bakery,Bank,Bar,Beach,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Bus Stop,Café,Caribbean Restaurant,Chaat Place,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Convenience Store,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Farm,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,Food Truck,French Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Home Service,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Light Rail Station,Malay Restaurant,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Modern European Restaurant,Multiplex,Museum,New American Restaurant,Nightclub,Noodle House,Optical Shop,Organic Grocery,Outdoor Supply Store,Outlet Store,Park,Performing Arts Venue,Pharmacy,Pie Shop,Pier,Pizza Place,Platform,Playground,Polish Restaurant,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,River,Salad Place,Sandwich Place,Scenic Lookout,Shopping Mall,Southern / Soul Food Restaurant,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Toll Plaza,Tourist Information Center,Train Station,Tram Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Barnet,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.030303,0.00,0.0,0.0,0.000000,0.0,0.060606,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.030303,0.000000,0.000000,0.0,0.030303,0.0,0.000000,0.0,0.121212,0.0,0.060606,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.0,0.000000,0.0,0.030303,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.060606,0.000000,0.030303,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.030303,0.0,0.0,0.0,0.060606,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.030303,0.0,0.030303,0.0,0.030303,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.060606,0.000000,0.060606,0.0,0.0,0.030303,0.000000,0.0,0.000000,0.000000,0.060606,0.000000,0.0,0.060606,0.0,0.000000,0.030303,0.0,0.030303,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.030303,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
1,Brent,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
2,Camden,0.0,0.05,0.0,0.000000,0.000000,0.000000,0.000000,0.05,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.05,0.000000,0.050000,0.000000,0.0,0.050000,0.0,0.000000,0.0,0.050000,0.0,0.050000,0.0,0.0

In [120]:
london_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Asian Restaurant,Bagel Shop,Bakery,Bank,Bar,Beach,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Bus Stop,Café,Caribbean Restaurant,Chaat Place,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Convenience Store,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Farm,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,Food Truck,French Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Home Service,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Light Rail Station,Malay Restaurant,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Modern European Restaurant,Multiplex,Museum,New American Restaurant,Nightclub,Noodle House,Optical Shop,Organic Grocery,Outdoor Supply Store,Outlet Store,Park,Performing Arts Venue,Pharmacy,Pie Shop,Pier,Pizza Place,Platform,Playground,Polish Restaurant,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,River,Salad Place,Sandwich Place,Scenic Lookout,Shopping Mall,Southern / Soul Food Restaurant,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Toll Plaza,Tourist Information Center,Train Station,Tram Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Barnet,0.000,0.00,0.00,0.000000,0.000000,0.000000,0.030303,0.00,0.000000,0.000000,0.000000,0.000000,0.060606,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.030303,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.00,0.121212,0.000000,0.060606,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.060606,0.000000,0.030303,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.00,0.000000,0.030303,0.000000,0.000000,0.00,0.060606,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.030303,0.00,0.030303,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.060606,0.000000,0.060606,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.060606,0.000000,0.000000,0.060606,0.000000,0.000000,0.030303,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Brent,0.000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

#### print top 5 venues in each neighborhood

In [121]:
num_top_venues = 5

for hood in london_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Barnet----
                venue  freq
0         Coffee Shop  0.12
1  Italian Restaurant  0.06
2           Bookstore  0.06
3   Convenience Store  0.06
4       Grocery Store  0.06


----Brent----
               venue  freq
0  Convenience Store   1.0
1  Afghan Restaurant   0.0
2           Pharmacy   0.0
3       Optical Shop   0.0
4    Organic Grocery   0.0


----Camden----
               venue  freq
0         Restaurant  0.10
1           Pharmacy  0.10
2         Donut Shop  0.05
3           Platform  0.05
4  Convenience Store  0.05


----Croydon----
                venue  freq
0                 Pub  0.21
1         Coffee Shop  0.17
2           Bookstore  0.04
3  Italian Restaurant  0.04
4         Gaming Cafe  0.04


----Ealing----
            venue  freq
0     Coffee Shop  0.11
1             Pub  0.06
2  Clothing Store  0.04
3        Platform  0.04
4          Bakery  0.03


----Enfield----
            venue  freq
0  Clothing Store  0.08
1     Coffee Shop  0.06
2            Café  0.06

In [122]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [128]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = london_grouped['Neighborhood']

for ind in np.arange(london_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barnet,Coffee Shop,Restaurant,Bookstore,Pharmacy,Convenience Store,Pub,Grocery Store,Park,Italian Restaurant,Modern European Restaurant
1,Brent,Convenience Store,Yoga Studio,Fast Food Restaurant,Furniture / Home Store,French Restaurant,Food Truck,Food,Flea Market,Fish & Chips Shop,Farm
2,Camden,Pharmacy,Restaurant,Light Rail Station,Park,Donut Shop,Pizza Place,Platform,Convenience Store,Coffee Shop,Sandwich Place
3,Croydon,Pub,Coffee Shop,Nightclub,Café,Spanish Restaurant,Gaming Cafe,Mediterranean Restaurant,Burger Joint,Malay Restaurant,Sushi Restaurant
4,Ealing,Coffee Shop,Pub,Platform,Clothing Store,Bakery,Café,Burger Joint,Park,Italian Restaurant,Bus Stop


In [129]:
type(neighborhoods_venues_sorted)

pandas.core.frame.DataFrame

#### apply knn to our london_grouped dataframe which have feature values in numbers for each venue type

In [130]:
# set number of clusters
kclusters = 4

london_grouped_clustering = london_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 1, 1, 1, 1, 1, 3, 1])

In [131]:
kmeans.labels_

array([1, 0, 1, 1, 1, 1, 1, 1, 3, 1, 1, 3, 1, 1, 1, 2])

In [133]:
london_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Asian Restaurant,Bagel Shop,Bakery,Bank,Bar,Beach,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Bus Stop,Café,Caribbean Restaurant,Chaat Place,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Convenience Store,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Farm,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,Food Truck,French Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Home Service,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Light Rail Station,Malay Restaurant,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Modern European Restaurant,Multiplex,Museum,New American Restaurant,Nightclub,Noodle House,Optical Shop,Organic Grocery,Outdoor Supply Store,Outlet Store,Park,Performing Arts Venue,Pharmacy,Pie Shop,Pier,Pizza Place,Platform,Playground,Polish Restaurant,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,River,Salad Place,Sandwich Place,Scenic Lookout,Shopping Mall,Southern / Soul Food Restaurant,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Toll Plaza,Tourist Information Center,Train Station,Tram Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Barnet,0.000,0.00,0.00,0.000000,0.000000,0.000000,0.030303,0.00,0.000000,0.000000,0.000000,0.000000,0.060606,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.030303,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.00,0.121212,0.000000,0.060606,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.060606,0.000000,0.030303,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.00,0.000000,0.030303,0.000000,0.000000,0.00,0.060606,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.030303,0.00,0.030303,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.060606,0.000000,0.060606,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.060606,0.000000,0.000000,0.060606,0.000000,0.000000,0.030303,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Brent,0.000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

#### add clustering labels

In [134]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

london_merged = london_data

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
london_merged = london_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

london_merged # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Newham,51.530000,0.029318,3,Café,Park,Fast Food Restaurant,Pub,French Restaurant,Food Truck,Food,Flea Market,Fish & Chips Shop,Yoga Studio
1,Redbridge,51.576320,0.045410,1,Hotel,Pub,Pizza Place,Eastern European Restaurant,Metro Station,Yoga Studio,Fish & Chips Shop,French Restaurant,Food Truck,Food
2,Brent,32.937346,-87.164718,0,Convenience Store,Yoga Studio,Fast Food Restaurant,Furniture / Home Store,French Restaurant,Food Truck,Food,Flea Market,Fish & Chips Shop,Farm
3,Harrow,51.596769,-0.337275,1,Indian Restaurant,Sandwich Place,Bus Stop,Coffee Shop,Fast Food Restaurant,Grocery Store,Home Service,Afghan Restaurant,Tunnel,Deli / Bodega
4,Ealing,51.512655,-0.305195,1,Coffee Shop,Pub,Platform,Clothing Store,Bakery,Café,Burger Joint,Park,Italian Restaurant,Bus Stop
5,Hounslow,51.468613,-0.361347,1,Fast Food Restaurant,Coffee Shop,Grocery Store,Clothing Store,Indian Restaurant,Hotel,Pharmacy,Bakery,Sandwich Place,Café
6,Hillingdon,51.542519,-0.448335,3,Park,Fast Food Restaurant,Pub,Chinese Restaurant,French Restaurant,Food Truck,Food,Flea Market,Fish & Chips Shop,Yoga Studio
7,Barnet,51.653090,-0.200226,1,Coffee Shop,Restaurant,Bookstore,Pharmacy,Convenience Store,Pub,Grocery Store,Park,Italian Restaurant,Modern European Restaurant
8,Croydon,51.371305,-0.101957,1,Pub,Coffee Shop,Nightclub,Café,Spanish Restaurant,Gaming Cafe,Mediterranean Restaurant,Burger Joint,Malay Restaurant,Sushi Restaurant
9,Merton,51.410803,-0.188099,1,Tram Station,Pub,Cricket Ground,Dessert Shop,Brazilian Restaurant,Sushi Restaurant,Flea Market,Thai Restaurant,Farm,Hardware Store


#### plot all the venues data which we got after segmenting them

In [135]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_merged['Latitude'], london_merged['Longitude'], london_merged['Neighborhood'], london_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [138]:
df['normalized_population']=df['Indian Population']/df['Indian Population'].max(axis=0)

In [140]:
london_merged['normalized_population']=df['normalized_population']

In [141]:
london_merged

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,normalized_population
0,Newham,51.530000,0.029318,3,Café,Park,Fast Food Restaurant,Pub,French Restaurant,Food Truck,Food,Flea Market,Fish & Chips Shop,Yoga Studio,0.673804
1,Redbridge,51.576320,0.045410,1,Hotel,Pub,Pizza Place,Eastern European Restaurant,Metro Station,Yoga Studio,Fish & Chips Shop,French Restaurant,Food Truck,Food,0.724176
2,Brent,32.937346,-87.164718,0,Convenience Store,Yoga Studio,Fast Food Restaurant,Furniture / Home Store,French Restaurant,Food Truck,Food,Flea Market,Fish & Chips Shop,Farm,0.920160
3,Harrow,51.596769,-0.337275,1,Indian Restaurant,Sandwich Place,Bus Stop,Coffee Shop,Fast Food Restaurant,Grocery Store,Home Service,Afghan Restaurant,Tunnel,Deli / Bodega,1.000000
4,Ealing,51.512655,-0.305195,1,Coffee Shop,Pub,Platform,Clothing Store,Bakery,Café,Burger Joint,Park,Italian Restaurant,Bus Stop,0.765095
5,Hounslow,51.468613,-0.361347,1,Fast Food Restaurant,Coffee Shop,Grocery Store,Clothing Store,Indian Restaurant,Hotel,Pharmacy,Bakery,Sandwich Place,Café,0.763842
6,Hillingdon,51.542519,-0.448335,3,Park,Fast Food Restaurant,Pub,Chinese Restaurant,French Restaurant,Food Truck,Food,Flea Market,Fish & Chips Shop,Yoga Studio,0.583575
7,Barnet,51.653090,-0.200226,1,Coffee Shop,Restaurant,Bookstore,Pharmacy,Convenience Store,Pub,Grocery Store,Park,Italian Restaurant,Modern European Restaurant,0.442816
8,Croydon,51.371305,-0.101957,1,Pub,Coffee Shop,Nightclub,Café,Spanish Restaurant,Gaming Cafe,Mediterranean Restaurant,Burger Joint,Malay Restaurant,Sushi Restaurant,0.391112
9,Merton,51.410803,-0.188099,1,Tram Station,Pub,Cricket Ground,Dessert Shop,Brazilian Restaurant,Sushi Restaurant,Flea Market,Thai Restaurant,Farm,Hardware Store,0.128563


In [142]:
london_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Asian Restaurant,Bagel Shop,Bakery,Bank,Bar,Beach,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Bus Stop,Café,Caribbean Restaurant,Chaat Place,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Convenience Store,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Farm,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,Food Truck,French Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Home Service,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Light Rail Station,Malay Restaurant,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Modern European Restaurant,Multiplex,Museum,New American Restaurant,Nightclub,Noodle House,Optical Shop,Organic Grocery,Outdoor Supply Store,Outlet Store,Park,Performing Arts Venue,Pharmacy,Pie Shop,Pier,Pizza Place,Platform,Playground,Polish Restaurant,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,River,Salad Place,Sandwich Place,Scenic Lookout,Shopping Mall,Southern / Soul Food Restaurant,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Toll Plaza,Tourist Information Center,Train Station,Tram Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Barnet,0.000,0.00,0.00,0.000000,0.000000,0.000000,0.030303,0.00,0.000000,0.000000,0.000000,0.000000,0.060606,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.030303,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.00,0.121212,0.000000,0.060606,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.060606,0.000000,0.030303,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.00,0.000000,0.030303,0.000000,0.000000,0.00,0.060606,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.030303,0.00,0.030303,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.060606,0.000000,0.060606,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.060606,0.000000,0.000000,0.060606,0.000000,0.000000,0.030303,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Brent,0.000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

### create a dataframe for neighborhood to get the ranking based on highest poossibility to open a restaurant

In [144]:
df_rank=london_grouped[['Neighborhood','Indian Restaurant']]

In [145]:
df_rank['normalized_population']=london_merged['normalized_population']

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [146]:
df_rank['Cluster Labels']=london_merged['Cluster Labels']

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [148]:
df_rank.head()

,Neighborhood,Indian Restaurant,normalized_population,Cluster Labels
0,Barnet,0.030303,0.673804,3
1,Brent,0.000000,0.724176,1
2,Camden,0.000000,0.920160,0
3,Croydon,0.041667,1.000000,1
4,Ealing,0.010638,0.765095,1


### give a positive weightage of Indian population which indicates thats higher success of restaurant
### give a negative weightage to already prsesent restauarnt which indicates lower chance of getting a crowd
### based on this caluclate the ranking

In [151]:
df_rank['ranking']=df_rank['normalized_population'] * 0.8 - df_rank['Indian Restaurant'] * 0.2

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [154]:
df_rank=df_rank.sort_values(by='ranking',ascending=False)

In [155]:
df_rank

,Neighborhood,Indian Restaurant,normalized_population,Cluster Labels,ranking
3,Croydon,0.041667,1.000000,1,0.791667
2,Camden,0.000000,0.920160,0,0.736128
4,Ealing,0.010638,0.765095,1,0.609948
5,Enfield,0.016129,0.763842,1,0.607848
1,Brent,0.000000,0.724176,1,0.579341
0,Barnet,0.030303,0.673804,3,0.532982
6,Greenwich,0.000000,0.583575,3,0.466860
7,Harrow,0.125000,0.442816,1,0.329253
8,Hillingdon,0.000000,0.391112,1,0.312890
11,Newham,0.000000,0.184739,1,0.147791


### Based on this ranking of suitability to open a restaurant, we can pick up the neighborhood in which there are higher population and less restaurant

#                                           Thank You